Requirements to run this:

* matplotlib
* numpy
* pandas
* seaborn
* patchworklib
    * `pip install patchworklib` (makes some bits of seaborn less awkward)
    * https://github.com/ponnhide/patchworklib


In [1]:
import sys
import subprocess
import pkg_resources

required = {"patchworklib","seaborn","numpy","matplotlib","scikit-learn","tqdm","pandas"}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    sys.stderr.write(f"ERROR: Missing the following required packages:\n{missing}")
    sys.stderr.write("Attempting to install missing dependencies to user directory")
    python = sys.executable
    subprocess.check_call([python, "-m", "pip","install",*missing,"--user"], stdout=subprocess.DEVNULL)


In [2]:

import numpy as np
import pandas as pd
from pandas import DataFrame
from typing import Iterable, Union, Dict, List, Tuple, NoReturn, Optional, Any
import inspect
import doctest
import sys

import uuid

import patchworklib as pw
import seaborn as sns
import matplotlib.pyplot as plt


from collections.abc import Hashable

<Figure size 72x72 with 0 Axes>

In [3]:
%matplotlib inline

sns.set_theme()

## Step 1: Loading the data

Before I can start analysing the data, I need to actually load it.
These functions (below) are intended to make that job a bit easier.

### 1.1: Some utility functions, to allow me to load the data in a way that makes some sense


In [4]:
def npz_to_dict(
        npz_filename: str,
        **kwargs
) -> Dict[str, np.ndarray]:
    """
    Puts the given NPZ file into a dictionary of {table name, ndarray}.
    :param npz_filename: the filename of the NPZ file that we want to put into a dictionary.
    :param kwargs: kwargs from https://numpy.org/doc/stable/reference/generated/numpy.load.html#numpy.load.
    DO NOT INCLUDE A 'mmap_mode' KWARG!!!
    :return: The data from the given npz file in a dictionary.
    """
    data_dict: Dict[str, np.ndarray] = {}
    if kwargs is None:
        kwargs = {}
    with np.load(npz_filename, mmap_mode="r",**kwargs) as npz:
        for f in npz.files:
            data_dict[f] = npz[f]
    return data_dict


def turn_01_columns_into_int(
        dataframe_to_edit: DataFrame,
) -> DataFrame:
    """
    Finds all of the columns that just contain values of 0 and 1,
    and converts all of those columns to ints.
    MODIFIES THE GIVEN DATAFRAME!
    :param dataframe_to_edit: the dataframe that is being edited
    :return: The modified dataframe.
    DOES NOT COPY THE GIVEN ORIGINAL DATAFRAME.

    >>> before: DataFrame = DataFrame.from_dict(data={"int01":[0,1,1,0],"flt01":[0.0, 1.0, 0.0, 1.0], "intNo": [-1,0,1,2], "fltNo":[-1.0, 0.0, 1.0, 2.0], "intNan": [0,1,None,0], "fltNan":[0.0,1.0,None,0.0]})
    >>> before_types = before.dtypes.values
    >>> after: DataFrame = turn_01_columns_into_int(before.copy())
    >>> after_types = after.dtypes.values
    >>> print(after_types[0])
    uint8
    >>> print(after_types[1])
    uint8
    >>> print(f"{before_types[2] == after_types[2]} {before_types[3] == after_types[3]} {before_types[4] == after_types[4]} {before_types[5] == after_types[5]}")
    True True True True
    """
    for c in dataframe_to_edit.columns:
        if dataframe_to_edit[c].dtype == np.uint8:
            continue
        if dataframe_to_edit[c].isin([0,1]).all():
            dataframe_to_edit[c] = dataframe_to_edit[c].astype(np.uint8)
    return dataframe_to_edit

if __name__ == "__main__":
    doctest.run_docstring_examples(turn_01_columns_into_int, globals())

def x_to_dataframe(
        x_data: np.ndarray,
        row_major = True,
        x_prefix: str = "x_"
) -> DataFrame:
    """
    Converts the 'x' ndarray into a pandas dataframe.
    :param x_data: the ndarray containing all of the data
    :param row_major: is this ndarray held in row-major order? [[item a data], [item b data], ... ]
    :param x_prefix: prefix to put on the names of all of the x columns
    :return: a dataframe holding the given x data.
    """
    if row_major:
        x_data: np.ndarray = x_data.T
    x_df: DataFrame = pd.DataFrame.from_dict({f"{x_prefix}{i}": x_data[i] for i in range(x_data.shape[0])})

    return turn_01_columns_into_int(x_df)



def add_everything_but_x_to_copy_of_dataframe(
        original_df: DataFrame,
        the_data_dict: Dict[str, np.ndarray],
        dont_add: Union[str, Iterable[str]]=frozenset('x')
) -> DataFrame:
    """
    Adds everything in the npz file apart from the given 'dont_add'
    tables to the dataframe.
    Assumes that these other tables have the same shape of (whatever, 1).
    :param original_df: the original dataframe that shall be copied and have stuff added to it.
    :param the_data_dict: The data file with the data to be added to the DataFrame
    :param dont_add: the identifier(s) of the columns that must not be added to the DataFrame.
    :return: a copy of the original dataframe, with the data from every table BESIDES the 'dont add' tables
    from the given file added to it.
    """

    the_df = original_df.copy()
    if dont_add in the_data_dict.keys():
        dont_add = [dont_add]
    for k, v in the_data_dict.items():
        if k in dont_add:
            continue
        the_df[k] = turn_01_columns_into_int(DataFrame(v))
    return the_df

#### 1.1.2: And some utility functions for producing graphs based on the datasets

In [5]:
def bin_finder(n: int, default_bins: int) -> int:

    if n <= default_bins:
        return n

    biggest_possible = np.floor(np.sqrt(n)).astype(int)

    potentials = np.arange(biggest_possible, stop=0, step=-1)
    divisor_indices = np.flatnonzero(n % potentials==0)

    print(potentials[divisor_indices])
    if divisor_indices.size > 0:
        print(f"{n} -> {n // potentials[divisor_indices[0]]}")
        return min(default_bins, n // potentials[divisor_indices[0]])
    return default_bins


def squareish_w_h(div_this: int) -> Tuple[int, int]:
    """
    Attempts to make a shape not entirely unlike a square
    :param div_this: int to divide
    :return: square-ish w/h
    """
    if np.sqrt(div_this) % 1 == 0:
        res = np.floor(np.sqrt(div_this))
        return res, res
    elif div_this & 1 == 0: # odd number
        return fallback_w_h(div_this)
    else:
        # if even, we work out which binary rectangle works best basically
        a: int = 1
        b: int = div_this
        while b > a and b & 1 == 0:
            b //= 2
            a *= 2
        if max(a, b) > min(a, b) * 2:
            return fallback_w_h(div_this)
        else:
            return tuple(*sorted((a, b), reverse=True))

def fallback_w_h(div_this: int) -> Tuple[int, int]:
    """
    Attempts to make a shape not entirely unlike a square
    :param div_this: int to divide
    :return: square-ish w/h
    """
    h = np.floor(np.sqrt(div_this))
    # get something relatively close to the square
    w = div_this//h

    i_w = w < h
    if w * h < div_this:
        if i_w:
            w += 1
        else:
            h += 1
        i_w = not i_w
    return w, h




def jointplot_maker(
        df: DataFrame,
        xlabel: str,
        ylabel: str,
        tlabel: str,
        wh: float = 5,
        debug: bool = False
) -> pw.Bricks:

    pw.param["margin"] = 0.1

    ax_x_label: str = f"{xlabel}{uuid.uuid4()}"
    #if debug:
    #    ax_x_label = f"{xlabel}{uuid.uuid4()}"

    ax1: pw.Brick = pw.Brick(f"{ax_x_label}_ax1",figsize=(wh, wh/4))

    if debug:
        print(type(ax1))

    sns.histplot(data=df, x=xlabel, hue=tlabel, kde=True, ax=ax1, element="step",stat="density",common_norm=False)
    ax1.spines["top"].set_visible(False)
    ax1.spines["right"].set_visible(False)

    ax2: pw.Brick = pw.Brick(f"{ax_x_label}_ax2",figsize=(wh/4, wh))
    sns.ecdfplot(data=df, y=ylabel, hue=tlabel, ax=ax2, complementary=True)
    ax2.spines["top"].set_visible(False)
    ax2.spines["right"].set_visible(False)

    ax3: pw.Brick = pw.Brick(f"{ax_x_label}_ax3",figsize=(3*wh/4, 3*wh/4))
    #sns.kdeplot(data=df, x=xlabel, y=ylabel, hue=tlabel, ax=ax3, alpha=.5)
    sns.scatterplot(data=df, x=xlabel, y=ylabel, hue=tlabel, ax=ax3, alpha=.5)
    # TODO: overlay kernel density over scatterplot?
    # TODO: kernel density might be better than scatter tbh

    ax1.set_xlim(ax3.get_xlim())
    ax1.set_xticks([])
    ax1.set_xlabel("")

    ax2.set_ylim(ax3.get_ylim())
    ax2.set_yticks([])
    ax2.set_ylabel("")

    ax13: pw.Bricks = ax1/ax3
    ax132: pw.Bricks = ax13[f"{ax_x_label}_ax3"]|ax2

    ax132.case.set_title(f"Distribution for {xlabel}")

    if debug:
        print(type(ax132))
        #print(type(ax132.savefig()))

    return ax132

def stdout_writer(write_this) -> NoReturn:
    sys.stdout.write(write_this)
    sys.stdout.flush()



def viz2(
        df: DataFrame, xlabels: List[Any],
        ylabel: Any, tlabel: Any,
        fig_subject: str, plt_wh: float = 5
) -> pw.Bricks:
    """
    Produces a series of graphs based on the given data
    :param df: the dataframe
    :param xlabels: column names for T
    :param ylabel: column name for Y
    :param tlabel: column name for T
    :param fig_subject: words for what the source of the data being shown is
    :param plt_wh: width/height for each individual plot in this figure
    :return: patchworklib Bricks object,
     showing the input data as a series of histograms
     (call .savefig() with no args to convert it to a viewable matplotlib figure)
    """

    #y_series: pd.Series = df[ylabel]
    #t_series: pd.Series = df[tlabel]

    #Y: np.ndarray = y_series.values
    #T: np.ndarray = t_series.values



    if fig_subject is None:
        fig_subject = ""
    else:
        fig_subject = f" {fig_subject}"

    print(xlabels)

    x_count: int = len(xlabels)

    w, h = squareish_w_h(x_count+2)

    #sns.FacetGrid = sns.FacetGrid(data=df, col="")

    #fig: plt.Figure = plt.Figure(
    #    figsize=(w*3, h*3)
    #)

    #axes: Tuple[plt.Axes] = fig.subplots(
    #    nrows=h, ncols=w, squeeze=True
    #)



    stdout_writer(f"making plots:\n"f"Todo: 00{'0'*x_count}\n"f"Done: ")
    
    axy_hist: pw.Brick = pw.Brick(f"{fig_subject}{uuid.uuid4()}_y_hist",figsize=(plt_wh, plt_wh))
    sns.histplot(data=df, x=ylabel, hue=tlabel, kde=True, ax=axy_hist, element="step",stat="density",common_norm=False)
    axy_hist.set_title("Histogram showing y values for treated/control groups")
    
    stdout_writer("\u2713")
    
    axy_dist: pw.Brick = pw.Brick(f"{fig_subject}{uuid.uuid4()}_y_dist",figsize=(plt_wh, plt_wh))
    sns.ecdfplot(data=df, x=ylabel, hue=tlabel, ax=axy_dist)
    axy_dist.set_title("Y values for treatment/control groups as a cumulative distribution")

    no_prior_rows: bool = True

    complete_figure: Optional[pw.Bricks] = None
    this_row: pw.Bricks = axy_hist | axy_dist

    stdout_writer("\u2713")
    
    for i in range(x_count):


        this_plot: pw.Bricks = jointplot_maker(
            df,
            xlabels[i],
            ylabel,
            tlabel,
            plt_wh
        )
        stdout_writer("\u2713")

        if (i+2) % w == 0:
            if no_prior_rows:
                complete_figure = this_row
                no_prior_rows = False
            else:
                complete_figure = complete_figure / this_row
            this_row = this_plot
        else:
            this_row = this_row | this_plot
            
    if no_prior_rows:
        complete_figure: pw.Bricks = this_row
    else:
        # TODO: pad out any unused space in the last row (so they aren't stupid big)
        complete_figure: pw.Bricks = complete_figure / this_row

    if complete_figure is None:
        raise RuntimeError("Somehow, nothing got added to the 'complete_figure', which is awkward.")


    stdout_writer("\n")

    #the_fig: pw.Bricks = fig_rows[0]

    #stdout_writer(f"Sticking the figures together:\n"
    #              f"Todo: {'0'*len(fig_rows)}\n"
    #              f"Done: \u2713")

    #for i in range(1, len(fig_rows)):
    #    the_fig /= fig_rows[i]
    #    stdout_writer("\u2713")

    #stdout_writer("\n")


    complete_figure.case.set_title(f"Graphs for {ylabel} given all X attributes and treatment groups in {fig_subject}")
    return complete_figure





def visualizer(df: DataFrame, xlabels: List[Any], ylabel: Any, tlabel: Any, fig_subject: str) -> plt.Figure:
    """
    Produces a series of graphs based on the given data
    :param df: the dataframe
    :param xlabels: column names for T
    :param ylabel: column name for Y
    :param tlabel: column name for T
    :param fig_subject: words for what the source of the data being shown is
    :return: a figure showing the input data as a series of histograms
    """

    y_series: pd.Series = df[ylabel]
    t_series: pd.Series = df[tlabel]

    Y: np.ndarray = y_series.values
    T: np.ndarray = t_series.values

    bins: int = bin_finder(Y.size, 20)



    if fig_subject is None:
        fig_subject = ""
    else:
        fig_subject = f" {fig_subject}"

    print(xlabels)

    y_t_str: Tuple[str, str] = ("Y", "T")
    y_t_np: Tuple[np.ndarray, np.ndarray] = (Y, T)
    y_t_bins: Tuple[int, int] = (
        min(bins,np.unique(Y).shape[0]),
        min(bins,np.unique(T).shape[0])
    )
    "This will be useful later on."

    y_treated: np.ndarray = Y[np.argwhere(T==1)]
    y_untreated: np.ndarray = Y[np.argwhere(T==0)]

    col_count: int = len(xlabels)
    fig: plt.Figure = plt.figure(figsize=(24, 1+(4*col_count)))
    subfigs: Tuple[plt.Figure, ...] = fig.subfigures(nrows=col_count, ncols=1)
    for i in range(col_count):

        sfig: plt.Figure = subfigs[i]
        label: str = xlabels[i]
        x_data: np.ndarray = df[label].to_numpy()


        this_bins = bin_finder(df[label].nunique(), 20)

        sfig.suptitle(f"Data for {label}")
        sfigs: Tuple[plt.Figure, plt.Figure] = sfig.subfigures(nrows=1,ncols=3,width_ratios=[1,1,3])

        hist1: plt.Figure = sfigs[0]
        hist1.suptitle(f"{label} quantities")
        hist_x_plt: plt.Axes = hist1.add_subplot()
        sns.histplot(data = x_data, stat="frequency", bins = this_bins, kde=True, ax=hist_x_plt)
        hist_x_plt.set_xlabel(f"{label}")
        hist_x_plt.set_ylabel("quantity")

        hist2: plt.Figure = sfigs[1]

        hist2_plt: plt.Axes = hist2.add_subplot()
        sns.kdeplot(
                data=df[[label, ylabel, tlabel]], x=label,y=ylabel,
                ax=hist2_plt, legend=True, hue=tlabel, fill=True,
                alpha=.5, #kind="kde"
        )
        hist2_plt.set_title(f"{label} and Y for treatment/control")
        hist2_plt.set_xlabel(f"{label}")
        hist2_plt.set_ylabel(f"outcome given {label}")

        scatters: plt.Figure = sfigs[2]

        x_treated: np.ndarray = x_data[np.argwhere(T==1)]
        x_untreated: np.ndarray = x_data[np.argwhere(T==0)]

        #hists.suptitle(f"More histograms for {label}")
        #hist_axes: Tuple[plt.Axes, plt.Axes, plt.Axes] = hists.subplots(nrows=1, ncols=2, sharex="all")


        scatters.suptitle(f"Scatter graphs for {label}")
        ax0, ax1, ax2 = scatters.subplots(nrows=1, ncols=3, sharex="all",sharey="all")
        ax0.set_xlabel(f"{label}")
        ax0.set_ylabel(f"Outcome from {label}")
        ax0.set_title("both")
        ax1.set_title("treated only")
        ax2.set_title("control only")
        ax0.scatter(x_treated, y_treated, c = "g", label = "Treated")
        ax1.scatter(x_treated, y_treated, c = "g")
        ax0.scatter(x_untreated, y_untreated, c = "r", label = "Control")
        ax2.scatter(x_untreated, y_untreated, c = "r")
        ax0.legend()

    fig.suptitle(f"X, Y, T visualizations all dimensions of X in{fig_subject}")
    return fig






### 1.2: Loading the IHPD dataset

Firstly, I need to load the dataset, and see what data is present.

In [6]:
ihdp_dict: Dict[str, np.ndarray] = npz_to_dict("ihdp.npz", allow_pickle = False)


for k,v in ihdp_dict.items():
    print(f"{k}: {v.shape}")

x: (747, 25)
t: (747, 1)
yf: (747, 1)
ycf: (747, 1)
ite: (747, 1)


#### 1.2.1: Extracting the 'x' column, and moving it to a dataframe

It looks like the 'x' data is a table of 747 rows and 25 columns.
This, by itself, is a bit difficult to digest, so I shall move that
into a dataframe, making it a bit easier to analyse (for now at least).

The following two code cells are just outputting an overview of the 'x'
data.

In [7]:
ihdp_df_x: DataFrame = x_to_dataframe(ihdp_dict['x'])
"""Dataframe holding the 'x' data for the IHDP dataset"""

ihdp_df_x.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24
0,1.397395,0.996346,-1.105624,-0.879606,0.308569,-1.023402,1,0,0,0,...,0,1,1,0,0,0,0,0,0,1
1,0.269033,0.196818,0.383828,0.161703,-0.629189,1.460832,1,0,1,0,...,1,1,1,1,0,0,0,0,0,0
2,1.051537,1.795874,-1.105624,0.161703,-0.629189,0.963985,1,0,1,1,...,1,1,1,0,0,0,0,0,0,1
3,0.662446,0.196818,-0.733261,-0.879606,0.371086,-0.692171,1,0,0,0,...,1,0,1,0,1,0,0,0,0,0
4,0.856992,1.795874,0.011465,-0.879606,0.558638,0.301522,0,1,1,0,...,1,0,1,1,0,0,0,0,0,0


In [8]:
print(ihdp_df_x.info())



ihdp_df_x.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 25 columns):
x_0     747 non-null float64
x_1     747 non-null float64
x_2     747 non-null float64
x_3     747 non-null float64
x_4     747 non-null float64
x_5     747 non-null float64
x_6     747 non-null uint8
x_7     747 non-null uint8
x_8     747 non-null uint8
x_9     747 non-null uint8
x_10    747 non-null uint8
x_11    747 non-null uint8
x_12    747 non-null uint8
x_13    747 non-null uint8
x_14    747 non-null uint8
x_15    747 non-null uint8
x_16    747 non-null uint8
x_17    747 non-null uint8
x_18    747 non-null uint8
x_19    747 non-null uint8
x_20    747 non-null uint8
x_21    747 non-null uint8
x_22    747 non-null uint8
x_23    747 non-null uint8
x_24    747 non-null uint8
dtypes: float64(6), uint8(19)
memory usage: 49.0 KB
None


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24
count,7.470000e+02,7.470000e+02,7.470000e+02,7.470000e+02,7.470000e+02,7.470000e+02,747.000000,747.000000,747.000000,747.000000,...,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000
mean,3.448082e-17,4.755975e-18,-2.377988e-17,-5.612051e-16,4.042579e-17,-7.633341e-16,0.514056,0.093708,0.520750,0.364123,...,0.959839,0.594378,0.963855,0.135207,0.135207,0.156627,0.081660,0.073628,0.128514,0.157965
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500137,0.291618,0.499904,0.481506,...,0.196467,0.491341,0.186775,0.342174,0.342174,0.363692,0.274029,0.261339,0.334886,0.364953
min,-2.731287e+00,-3.800823e+00,-1.850350e+00,-8.796060e-01,-5.130428e+00,-1.851480e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.669461e-01,-6.027100e-01,-7.332610e-01,-8.796060e-01,-5.666720e-01,-8.577868e-01,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.652752e-01,1.968181e-01,-3.608980e-01,1.617025e-01,1.210172e-01,-2.970882e-02,1.000000,0.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.137593e-01,5.965822e-01,7.561910e-01,1.617025e-01,6.836721e-01,6.327536e-01,1.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.505476e+00,2.595403e+00,2.990369e+00,2.244320e+00,2.371637e+00,2.951372e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
print(ihdp_df_x.nunique())

print("\nThe actual value counts for x_3")
print(ihdp_df_x["x_3"].value_counts())

x_0     214
x_1      46
x_2      14
x_3       4
x_4      87
x_5      30
x_6       2
x_7       2
x_8       2
x_9       2
x_10      2
x_11      2
x_12      2
x_13      2
x_14      2
x_15      2
x_16      2
x_17      2
x_18      2
x_19      2
x_20      2
x_21      2
x_22      2
x_23      2
x_24      2
dtype: int64

The actual value counts for x_3
-0.879606    346
 0.161703    236
 1.203011    100
 2.244320     65
Name: x_3, dtype: int64


Looking at `x_3`, and how all of the values in it are one of four distinct values,
 all of which are painfully close to being int values, I **really** want to just
 convert them into raw ints, which would be rather trivial to do,
 as follows (converting -0.879606 to 0, 0.161703 to 1, 1.203011 to 2,  2.244320 to 3, etc):

```python
import pandas as pd
import numpy as np
def x3_to_int(df: pd.DataFrame) -> pd.DataFrame:
    x3 = df["x_3"]
    x3_vals = [*x3.value_counts().keys()]
    x3_vals.sort()
    x3_diff = x3_vals[2] - x3_vals[1]
    x3 += x3_diff
    x3 /= x3_diff
    df["x_3"] = x3.astype(np.int8)
    return df
```

However, I will refrain from doing this. This is because I don't actually know what
the data in `x_3` is supposed to be. If I knew, with certainty, that `x_3`
would only ever be one of those for values (or, failing that, would always
be a value from a continuous range of values with a consistent
gap of `1.041308` between each value), I would be applying that bit of
scaling to `x_3`. But, as mentioned earlier, I don't. So I can't. Which is a shame.



#### 1.2.2: Extracting the rest of the IHDP dataset, before adding it to a dataframe

I shall copy the X data into a new dataframe (ensuring that I retain a backup copy of the dataframe holding
just the X data), and add the rest of the IHDP data to that new dataframe,
meaning that I'll be able to get a full overview of IHDP.


In [10]:
ihdp_df: DataFrame = add_everything_but_x_to_copy_of_dataframe(
    ihdp_df_x.copy(),
    ihdp_dict,
    "x"
)
"""Dataframe holding the entirety of the IHDP dataset"""

ihdp_df.head()


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_19,x_20,x_21,x_22,x_23,x_24,t,yf,ycf,ite
0,1.397395,0.996346,-1.105624,-0.879606,0.308569,-1.023402,1,0,0,0,...,0,0,0,0,0,1,1,4.771232,-0.298509,4.657928
1,0.269033,0.196818,0.383828,0.161703,-0.629189,1.460832,1,0,1,0,...,0,0,0,0,0,0,0,2.956273,5.783770,3.428604
2,1.051537,1.795874,-1.105624,0.161703,-0.629189,0.963985,1,0,1,1,...,0,0,0,0,0,1,0,4.164164,7.055789,3.658195
3,0.662446,0.196818,-0.733261,-0.879606,0.371086,-0.692171,1,0,0,0,...,1,0,0,0,0,0,1,6.172307,1.379697,4.585505
4,0.856992,1.795874,0.011465,-0.879606,0.558638,0.301522,0,1,1,0,...,0,0,0,0,0,0,1,7.834469,2.747986,4.265591


In [11]:
print("info about the full dataset")
print(ihdp_df.info())

print("\ntreatment info for the full dataset")
print(ihdp_df["t"].value_counts())

print("\ndescribing the dataset")
ihdp_df.describe()

info about the full dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 29 columns):
x_0     747 non-null float64
x_1     747 non-null float64
x_2     747 non-null float64
x_3     747 non-null float64
x_4     747 non-null float64
x_5     747 non-null float64
x_6     747 non-null uint8
x_7     747 non-null uint8
x_8     747 non-null uint8
x_9     747 non-null uint8
x_10    747 non-null uint8
x_11    747 non-null uint8
x_12    747 non-null uint8
x_13    747 non-null uint8
x_14    747 non-null uint8
x_15    747 non-null uint8
x_16    747 non-null uint8
x_17    747 non-null uint8
x_18    747 non-null uint8
x_19    747 non-null uint8
x_20    747 non-null uint8
x_21    747 non-null uint8
x_22    747 non-null uint8
x_23    747 non-null uint8
x_24    747 non-null uint8
t       747 non-null uint8
yf      747 non-null float64
ycf     747 non-null float64
ite     747 non-null float64
dtypes: float64(9), uint8(20)
memory usage: 67.2 KB
None

treatmen

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_19,x_20,x_21,x_22,x_23,x_24,t,yf,ycf,ite
count,7.470000e+02,7.470000e+02,7.470000e+02,7.470000e+02,7.470000e+02,7.470000e+02,747.000000,747.000000,747.000000,747.000000,...,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000
mean,3.448082e-17,4.755975e-18,-2.377988e-17,-5.612051e-16,4.042579e-17,-7.633341e-16,0.514056,0.093708,0.520750,0.364123,...,0.135207,0.156627,0.081660,0.073628,0.128514,0.157965,0.186078,3.159538,5.696107,4.016067
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500137,0.291618,0.499904,0.481506,...,0.342174,0.363692,0.274029,0.261339,0.334886,0.364953,0.389430,2.179956,1.980121,0.859736
min,-2.731287e+00,-3.800823e+00,-1.850350e+00,-8.796060e-01,-5.130428e+00,-1.851480e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.543902,-1.037628,-1.866989
25%,-6.669461e-01,-6.027100e-01,-7.332610e-01,-8.796060e-01,-5.666720e-01,-8.577868e-01,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.626779,5.053598,3.775936
50%,1.652752e-01,1.968181e-01,-3.608980e-01,1.617025e-01,1.210172e-01,-2.970882e-02,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.577294,6.209686,4.304433
75%,8.137593e-01,5.965822e-01,7.561910e-01,1.617025e-01,6.836721e-01,6.327536e-01,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.494637,6.948922,4.569454
max,1.505476e+00,2.595403e+00,2.990369e+00,2.244320e+00,2.371637e+00,2.951372e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.268228,10.171004,4.670125


#### 1.2.3: Visualizing the IHDP dataset

In [12]:
# jointplot_maker(ihdp_df[["x_0","yf","t"]],"x_0","yf","t", debug=True).savefig()

In [13]:
#visualizer(ihdp_df, [*ihdp_df_x.columns] , "yf", "t",fig_subject="IHDP").show()

ihdp_fig: plt.Figure = viz2(ihdp_df, [*ihdp_df_x.columns], "yf","t", fig_subject="IHDP").savefig()

ihdp_fig

['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24']
making plots:
Todo: 000000000000000000000000000
Done: ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓


In [14]:
ihdp_fig.savefig("ihdp_graphs.pdf", format="pdf", bbox_inches="tight")


#### 1.2.4: Discussing the IHDP dataset

We can see that most of the X measures in this dataset are boolean 'yes/no' values (with only x values 0-5
being continuous values). This implies that we don't need to perform any scaling on these latter 19 values to keep
them in a manageable range, however, we may need to consider


### 1.3: Loading the JOBS dataset

As above, but for the JOBS dataset instead.

In [15]:
jobs_dict: Dict[str, np.ndarray] = npz_to_dict("jobs.npz")


for k,v in jobs_dict.items():
    print(f"{k}: {v.shape}")

x: (3212, 17)
t: (3212, 1)
y: (3212, 1)
e: (3212, 1)


#### 1.3.1: Loading the X data from JOBS

In [16]:
jobs_df_x: DataFrame = x_to_dataframe(jobs_dict['x'])

jobs_df_x.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16
0,-0.614282,1.464727,0,0,1,0,2.393250,2.746196,-0.653311,-0.656913,1.627531,2.462337,2.937244,0,0,2.843909,0
1,-0.802463,0.101835,0,0,1,0,0.109885,0.498271,-0.785284,-0.743407,-0.022502,-0.177193,0.082537,0,0,0.038422,0
2,-0.896553,-0.238888,1,0,1,1,-0.085212,-0.148097,-0.847312,-0.781606,-0.361348,-0.286221,-0.303615,0,0,-0.191304,0
3,-0.896553,-0.238888,0,0,0,1,0.405581,0.325594,-0.847312,-0.781606,-0.361348,0.023020,-0.039630,0,0,0.173108,0
4,0.138440,-1.601779,1,0,1,1,-0.722531,-0.212734,-0.019840,-0.156019,-1.422084,-0.514563,-0.331552,0,0,-0.779227,0


In [17]:
print(jobs_df_x.info())

jobs_df_x.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3212 entries, 0 to 3211
Data columns (total 17 columns):
x_0     3212 non-null float64
x_1     3212 non-null float64
x_2     3212 non-null uint8
x_3     3212 non-null uint8
x_4     3212 non-null uint8
x_5     3212 non-null uint8
x_6     3212 non-null float64
x_7     3212 non-null float64
x_8     3212 non-null float64
x_9     3212 non-null float64
x_10    3212 non-null float64
x_11    3212 non-null float64
x_12    3212 non-null float64
x_13    3212 non-null uint8
x_14    3212 non-null uint8
x_15    3212 non-null float64
x_16    3212 non-null uint8
dtypes: float64(10), uint8(7)
memory usage: 273.0 KB
None


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16
count,3.212000e+03,3.212000e+03,3212.000000,3212.000000,3212.000000,3212.000000,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3212.000000,3212.000000,3.212000e+03,3212.000000
mean,2.013057e-16,-1.880328e-16,0.374222,0.048879,0.707970,0.411893,3.318226e-17,-9.069817e-17,-1.106075e-17,1.763748e-14,1.437898e-17,-3.539441e-17,2.035178e-16,0.168742,0.167497,1.283047e-16,0.010897
std,1.000000e+00,1.000000e+00,0.483997,0.215649,0.454767,0.492253,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.374582,0.373477,1.000000e+00,0.103833
min,-1.461095e+00,-3.986840e+00,0.000000,0.000000,0.000000,0.000000,-1.147411e+00,-1.110467e+00,-1.164047e+00,-9.488316e-01,-2.143974e+00,-5.580568e-01,-5.188151e-01,0.000000,0.000000,-9.799811e-01,0.000000
25%,-8.024627e-01,-5.796108e-01,0.000000,0.000000,0.000000,0.000000,-8.923385e-01,-9.139141e-01,-7.852843e-01,-7.434072e-01,-6.707297e-01,-5.423814e-01,-5.098380e-01,0.000000,0.000000,-8.142367e-01,0.000000
50%,-3.320111e-01,1.018350e-01,0.000000,0.000000,1.000000,0.000000,-8.521244e-02,-8.346011e-02,-4.355556e-01,-4.998533e-01,-2.250229e-02,-2.862214e-01,-2.737369e-01,0.000000,0.000000,-2.035020e-01,0.000000
75%,7.970728e-01,4.425579e-01,1.000000,0.000000,1.000000,1.000000,5.506613e-01,5.629081e-01,6.730184e-01,5.267384e-01,3.458087e-01,1.366588e-01,1.318304e-01,0.000000,0.000000,4.539777e-01,0.000000
max,2.114337e+00,1.805449e+00,1.000000,1.000000,1.000000,1.000000,8.751987e+00,1.016507e+01,2.446737e+00,2.773336e+00,2.113702e+00,2.305284e+01,2.902264e+01,1.000000,1.000000,7.136030e+00,1.000000


In [18]:

print(jobs_df_x.nunique())

x_0       39
x_1       17
x_2        2
x_3        2
x_4        2
x_5        2
x_6      799
x_7      843
x_8       39
x_9       39
x_10      17
x_11     799
x_12     843
x_13       2
x_14       2
x_15    1373
x_16       2
dtype: int64


#### 1.3.2: Loading the remaining data from JOBS

In [19]:
jobs_df: DataFrame = add_everything_but_x_to_copy_of_dataframe(
    jobs_df_x.copy(),
    jobs_dict,
    "x"
)

jobs_df.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,t,y,e
0,-0.614282,1.464727,0,0,1,0,2.393250,2.746196,-0.653311,-0.656913,1.627531,2.462337,2.937244,0,0,2.843909,0,0,1,0
1,-0.802463,0.101835,0,0,1,0,0.109885,0.498271,-0.785284,-0.743407,-0.022502,-0.177193,0.082537,0,0,0.038422,0,0,1,0
2,-0.896553,-0.238888,1,0,1,1,-0.085212,-0.148097,-0.847312,-0.781606,-0.361348,-0.286221,-0.303615,0,0,-0.191304,0,0,1,0
3,-0.896553,-0.238888,0,0,0,1,0.405581,0.325594,-0.847312,-0.781606,-0.361348,0.023020,-0.039630,0,0,0.173108,0,0,1,1
4,0.138440,-1.601779,1,0,1,1,-0.722531,-0.212734,-0.019840,-0.156019,-1.422084,-0.514563,-0.331552,0,0,-0.779227,0,0,1,0


In [20]:
print(jobs_df.info())

jobs_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3212 entries, 0 to 3211
Data columns (total 20 columns):
x_0     3212 non-null float64
x_1     3212 non-null float64
x_2     3212 non-null uint8
x_3     3212 non-null uint8
x_4     3212 non-null uint8
x_5     3212 non-null uint8
x_6     3212 non-null float64
x_7     3212 non-null float64
x_8     3212 non-null float64
x_9     3212 non-null float64
x_10    3212 non-null float64
x_11    3212 non-null float64
x_12    3212 non-null float64
x_13    3212 non-null uint8
x_14    3212 non-null uint8
x_15    3212 non-null float64
x_16    3212 non-null uint8
t       3212 non-null uint8
y       3212 non-null uint8
e       3212 non-null uint8
dtypes: float64(10), uint8(10)
memory usage: 282.4 KB
None


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,t,y,e
count,3.212000e+03,3.212000e+03,3212.000000,3212.000000,3212.000000,3212.000000,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3.212000e+03,3212.000000,3212.000000,3.212000e+03,3212.000000,3212.000000,3212.000000,3212.000000
mean,2.013057e-16,-1.880328e-16,0.374222,0.048879,0.707970,0.411893,3.318226e-17,-9.069817e-17,-1.106075e-17,1.763748e-14,1.437898e-17,-3.539441e-17,2.035178e-16,0.168742,0.167497,1.283047e-16,0.010897,0.092466,0.849938,0.224782
std,1.000000e+00,1.000000e+00,0.483997,0.215649,0.454767,0.492253,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.374582,0.373477,1.000000e+00,0.103833,0.289727,0.357188,0.417504
min,-1.461095e+00,-3.986840e+00,0.000000,0.000000,0.000000,0.000000,-1.147411e+00,-1.110467e+00,-1.164047e+00,-9.488316e-01,-2.143974e+00,-5.580568e-01,-5.188151e-01,0.000000,0.000000,-9.799811e-01,0.000000,0.000000,0.000000,0.000000
25%,-8.024627e-01,-5.796108e-01,0.000000,0.000000,0.000000,0.000000,-8.923385e-01,-9.139141e-01,-7.852843e-01,-7.434072e-01,-6.707297e-01,-5.423814e-01,-5.098380e-01,0.000000,0.000000,-8.142367e-01,0.000000,0.000000,1.000000,0.000000
50%,-3.320111e-01,1.018350e-01,0.000000,0.000000,1.000000,0.000000,-8.521244e-02,-8.346011e-02,-4.355556e-01,-4.998533e-01,-2.250229e-02,-2.862214e-01,-2.737369e-01,0.000000,0.000000,-2.035020e-01,0.000000,0.000000,1.000000,0.000000
75%,7.970728e-01,4.425579e-01,1.000000,0.000000,1.000000,1.000000,5.506613e-01,5.629081e-01,6.730184e-01,5.267384e-01,3.458087e-01,1.366588e-01,1.318304e-01,0.000000,0.000000,4.539777e-01,0.000000,0.000000,1.000000,0.000000
max,2.114337e+00,1.805449e+00,1.000000,1.000000,1.000000,1.000000,8.751987e+00,1.016507e+01,2.446737e+00,2.773336e+00,2.113702e+00,2.305284e+01,2.902264e+01,1.000000,1.000000,7.136030e+00,1.000000,1.000000,1.000000,1.000000


In [21]:
jobs_graphs: plt.Figure = viz2(jobs_df,[*jobs_df_x.columns],"y","t","Jobs").savefig()

jobs_graphs

['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16']
making plots:
Todo: 0000000000000000000
Done: ✓✓✓✓✓✓✓✓✓✓✓


In [22]:
jobs_graphs.savefig("jobs_graphs.pdf", format="pdf", bbox_inches="tight")

#### 1.3.3: Discussing the JOBS dataset
